In [1]:
from connections import bb_session
bb_s = bb_session()

# Get Course Memberships
The LTI Tool does the following
* Pull membership information
* Create a group for each unique child course
* Enroll members in groups 

In [17]:
parentCourse = 'TST-Merge01'
memberships = bb_s.get(f'v1/courses/courseId:{parentCourse}/users').json()['results']
memberships = [(membership['userId'], membership.get('childCourseId', membership.get('courseId'))) 
                 for membership in memberships 
                 if membership['courseRoleId'] == 'Student']

In [18]:
memberships

[('_343656_1', '_273814_1'),
 ('_428988_1', '_273814_1'),
 ('_25264_1', '_273813_1'),
 ('_383704_1', '_273813_1')]

In [19]:
bb_s.get(f'v1/courses/courseId:{parentCourse}/users').json()['results']

[{'userId': '_343656_1',
  'courseId': '_273813_1',
  'childCourseId': '_273814_1',
  'dataSourceId': '_2_1',
  'created': '2020-10-01T15:15:30.000Z',
  'availability': {'available': 'Yes'},
  'courseRoleId': 'Student'},
 {'userId': '_428988_1',
  'courseId': '_273813_1',
  'childCourseId': '_273814_1',
  'dataSourceId': '_2_1',
  'created': '2020-10-01T15:15:30.000Z',
  'availability': {'available': 'Yes'},
  'courseRoleId': 'Student'},
 {'userId': '_25264_1',
  'courseId': '_273813_1',
  'dataSourceId': '_2_1',
  'created': '2020-10-01T15:15:30.000Z',
  'availability': {'available': 'Yes'},
  'courseRoleId': 'Student'},
 {'userId': '_383704_1',
  'courseId': '_273813_1',
  'dataSourceId': '_2_1',
  'created': '2020-10-01T15:15:30.000Z',
  'availability': {'available': 'Yes'},
  'courseRoleId': 'Student'},
 {'userId': '_339517_1',
  'courseId': '_273813_1',
  'dataSourceId': '_2_1',
  'created': '2020-10-01T15:01:16.000Z',
  'availability': {'available': 'Yes'},
  'courseRoleId': 'Ins